# CNN Transfer Learning on Cats-Dogs Classification

## CIML Summer Institute, UC San Diego

VGG16 trained on ImageNet data is used as a pre-trained model from which to extract features. Features are then passed through fully connected layers to classify cats vs. dogs.

Adapted from tensorflow tutorials
(https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb)

---



### Setup

In [1]:
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from sklearn.metrics import classification_report 
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import time

In [2]:
print("Tensorflow version:",tf.__version__)
!python --version
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

Tensorflow version: 2.3.0
Python 3.6.9
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Sat Jun 19 13:29:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:18:00.0 Off |                    0 |
| N/A   38C    P0    64W / 300W |      3MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+------

In [3]:
# Set logging level
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
# Set random generator seed
seed = 1234

# Disable hash randomization by specifying the value 0.
os.environ['PYTHONHASHSEED'] = '0'

# Set numpy random generator
np.random.seed(seed)

# Set python built-in random generator
random.seed(seed)

# Set tf global random seed
tf.random.set_seed(seed)

# Set tensorflow graph-level random seed
tf.compat.v1.random.set_random_seed(seed)

#potential randomness from CUDNN
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC']= '1'

### Set image dimensions, location of images

In [5]:
# Image dimensions
img_width, img_height = 150, 150
IMG_SIZE = (img_width,img_height)
IMG_SHAPE = IMG_SIZE + (3,)

# Location of images
train_data_dir = '../data/catsVsDogs/train'
validation_data_dir = '../data/catsVsDogs/val'
test_data_dir = '../data/catsVsDogs/test'

# Batch size
BATCH_SIZE = 16

### Prepare data

In [6]:
# Data augmentation setup
datagen = ImageDataGenerator(rescale = 1. / 255)

# Set up generator to read images found in subfolders of training data directory,
# and indefinitely generate batches of image data (scaled).  This is for training data.
train_generator = datagen.flow_from_directory(train_data_dir,target_size=IMG_SIZE,
                                              batch_size = BATCH_SIZE, class_mode='binary', 
                                              shuffle = True, seed = seed)           

# Set up generator to generate batched of validation data for model
validation_generator = datagen.flow_from_directory(validation_data_dir,target_size=IMG_SIZE,
                                                   batch_size = BATCH_SIZE,class_mode='binary',
                                                   shuffle = False, seed = seed)
# Set up generator to generate batched of test data for model
test_generator = datagen.flow_from_directory(test_data_dir, target_size=IMG_SIZE,
                                             batch_size = BATCH_SIZE, class_mode='binary',
                                             shuffle = False, seed = seed)

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


### Load pre-trained VGG16 model
Pre-trained model is used as feature extractor, so all weights are frozen.

In [7]:
# Load VGG16 model's Imagenet weights not including the top fully connected layers
base_model = applications.VGG16(include_top = False, weights = 'imagenet', 
                                input_shape=IMG_SHAPE)
# Freeze all weights of VGG16 model
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

### Create top model

Create fully connected layers to put on top of pre-trained VGG16.  

In [8]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


### Train model
Train weights in top model

In [ ]:
EPOCHS = 20

# Compile model with Adam optimizer with slow learning rate,
# Binary Cross-Entropy loss function and Accuracy metric
model.compile(optimizer=optimizers.Adam(lr=0.0001),
              loss= losses.BinaryCrossentropy(),metrics=['accuracy'])

# Early Stopping to avoid overfitting and ModelCheckpoint to save the best model
checkpoint_path = 'tmp/checkpoint'
callbacks = [EarlyStopping(monitor='val_loss' ,patience=3, min_delta=0.001, mode='min'),
             ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                             mode = 'min', save_best_only = True, 
                             save_weights_only=True)]

In [ ]:
%%time

train_history = model.fit(train_generator,epochs=EPOCHS, 
                          validation_data=validation_generator, 
                          callbacks=callbacks)

Epoch 1/20
 47/125 [==========>...................] - ETA: 2s - loss: 0.5634 - accuracy: 0.6968

In [ ]:
# Load the best model that was saved using ModelCheckpoint
model.load_weights(checkpoint_path)

In [ ]:
# Save model weights 
model.save("weights_from_feature_extract")

In [ ]:
# Plot train and validation history
fig, axs = plt.subplots(1,2, figsize= (20,5))
axs[0].plot(train_history.history['loss'])
axs[0].plot(train_history.history['val_loss'])
axs[0].set_title("Train, Val loss history")
axs[0].set_xlabel("Epoch")
axs[0].legend(["Train Loss","Val Loss"])

axs[1].plot(train_history.history['accuracy'])
axs[1].plot(train_history.history['val_accuracy'])
axs[1].set_title("Train, Val Accuracy history")
axs[1].set_xlabel("Epoch")
axs[1].legend(["Train Accuracy","Val Accuracy"])

### Evaluate model

In [ ]:
_, train_accuracy = model.evaluate(train_generator)
print("Train data accuracy:", train_accuracy)

_, test_accuracy = model.evaluate(test_generator)
print("Test data accuracy:", test_accuracy)

In [ ]:
# Get predicted value and the ground truth value of test data
pred = model.predict_classes(test_generator)
true = test_generator.classes

In [ ]:
# Classification report for test data
print(classification_report(y_true= true, y_pred = pred, target_names=['cats', 'dogs']))

### Perform inference on test images

In [ ]:
def image_loader(img_file):
    img = load_img(img_file, target_size = (img_width, img_height))
    imgplot = plt.imshow(img)
    plt.show()
    img = img_to_array(img) / 255
    img = np.expand_dims(img, axis = 0) #model input is (1,150,150,3)
    return img

In [ ]:
img = image_loader('../data/catsVsDogs/test/cats/cat.1070.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
img = image_loader('../data/catsVsDogs/test/dogs/dog.1233.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
img = image_loader('../data/catsVsDogs/test/cats/cat.1195.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
img = image_loader('../data/catsVsDogs/test/cats/cat.1011.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)